# Imports

In [2]:
import math
import random
from collections import deque
import heapq
import csv
import os
import matplotlib.pyplot as plt
import pandas as pd
import ast

ModuleNotFoundError: No module named 'matplotlib'

#Criação das pastas que serão gravadas os arquivos txt e csv

In [ ]:
if not os.path.exists("experiment_txt"):
    os.mkdir("experiment_txt")
if not os.path.exists("experiment_csv"):
    os.mkdir("experiment_csv")

# Constantes

In [ ]:
# Constantes para os limites do mapa
MAP_LIMIT = 30

# Estrutura Node:

In [ ]:
class Node:
    def __init__(self, position, parent=None, cost=0, depth=0):
        """Representa um nó na árvore de busca."""
        self.position = position  # Coordenadas (x, y)
        self.parent = parent      # Nó pai
        self.cost = cost          # Custo acumulado
        self.depth = depth        # Profundidade na árvore

    def __lt__(self, other):
        return self.cost < other.cost

# Movimentação

In [ ]:
def move_left(x, y): # f1
    return (x - 1, y) if x > 0 else None

def move_right(x, y): # f2
    return (x + 1, y) if x < MAP_LIMIT else None

def move_down(x, y): # f3
    return (x, y - 1) if y > 0 else None

def move_up(x, y): # f4
    return (x, y + 1) if y < MAP_LIMIT else None

# Mapeamento de operadores
OPERATORS = {
    "LEFT": move_left,
    "RIGHT": move_right,
    "UP": move_up,
    "DOWN": move_down
}

# Função de custo

In [ ]:
def cost_c1(action=None):
    return 10

def cost_c2(action):
    return 10 if action in ["UP", "DOWN"] else 15

def cost_c3(action, steps):
    return 10 if action in ["UP", "DOWN"] else 10 + (abs(5 - steps) % 6)

def cost_c4(action, steps):
    return 10 if action in ["UP", "DOWN"] else 5 + (abs(10 - steps) % 11)

# Heurísticas

In [ ]:
def euclidean_heuristic(pos1, pos2):
    x1, y1 = pos1
    x2, y2 = pos2
    return 10 * math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def manhattan_heuristic(pos1, pos2):
    x1, y1 = pos1
    x2, y2 = pos2
    return 10 * (abs(x1 - x2) + abs(y1 - y2))

# Função auxiliar para reconstruir o caminho
def reconstruct_path(node):
    path = []
    while node:
        path.append(node.position)
        node = node.parent
    return path[::-1]

# **Função de busca**


---
Executa um algoritmo de busca genérico.
  
*   start: ponto inicial (x, y).
*   goal: ponto objetivo (x, y).
*   algorithm: tipo de algoritmo (e.g., "BFS", "DFS", "UCS", "GREEDY", "A*").
*   cost_function: função de custo (e.g., cost_c1, cost_c2).
*   heuristic: função heurística (necessária para GREEDY e A*).



In [ ]:

def search_algorithm(start, goal, algorithm, cost_function, heuristic=None):

    if algorithm == "BFS":
        frontier = deque([Node(start)])
        visited = set()
        nodes_generated, nodes_visited = 0, 0

        while frontier:
            current = frontier.popleft()
            nodes_visited += 1

            if current.position == goal:
                return reconstruct_path(current), current.cost, nodes_generated, nodes_visited

            visited.add(current.position)

            for action, move in OPERATORS.items():
                neighbor = move(*current.position)
                if neighbor and neighbor not in visited:
                    nodes_generated += 1
                    visited.add(neighbor)
                    if cost_function in [cost_c3, cost_c4]:
                        new_cost = current.cost + cost_function(action, current.depth + 1)
                    else:
                        new_cost = current.cost + cost_function(action)
                # Adiciona o novo nó à fila
                    frontier.append(Node(neighbor, current, new_cost, current.depth + 1))

        return None, float("inf"), nodes_generated, nodes_visited

    elif algorithm == "DFS":
        frontier = [Node(start)]  # Usamos uma pilha
        visited = set()
        nodes_generated, nodes_visited = 0, 0

        while frontier:
            current = frontier.pop()  # Remove o último elemento da pilha
            nodes_visited += 1

            if current.position == goal:
                return reconstruct_path(current), current.cost, nodes_generated, nodes_visited

            if current.position not in visited:
                visited.add(current.position)

                for action, move in OPERATORS.items():
                    neighbor = move(*current.position)
                    if neighbor and neighbor not in visited:
                        nodes_generated += 1
                        new_cost = current.cost + (cost_function(action, current.depth + 1) if cost_function in [cost_c3, cost_c4] else cost_function(action))
                        frontier.append(Node(neighbor, current, new_cost, current.depth + 1))

        return None, float("inf"), nodes_generated, nodes_visited

    elif algorithm == "UCS":
        frontier = []  # Fila de prioridade
        heapq.heappush(frontier, Node(start, cost=0))  # Adiciona o nó inicial com custo 0
        visited = {}  # Dicionário para armazenar o menor custo para cada nó
        nodes_generated, nodes_visited = 0, 0

        while frontier:
            current = heapq.heappop(frontier)  # Remove o nó com menor custo
            nodes_visited += 1

            # Verifica se alcançamos o objetivo
            if current.position == goal:
                return reconstruct_path(current), current.cost, nodes_generated, nodes_visited

            # Checa se o nó atual deve ser explorado (custo menor do que já visto)
            if current.position not in visited or current.cost < visited[current.position]:
                visited[current.position] = current.cost

                # Expande os vizinhos
                for action, move in OPERATORS.items():
                    neighbor = move(*current.position)
                    if neighbor:
                        nodes_generated += 1
                        new_cost = current.cost + (cost_function(action, current.depth + 1) if cost_function in [cost_c3, cost_c4] else cost_function(action))
                        heapq.heappush(frontier, Node(neighbor, current, new_cost, current.depth + 1))

        return None, float("inf"), nodes_generated, nodes_visited

    # Outros algoritmos serão adicionados aqui (GREEDY, A*)
    elif algorithm == "GREEDY":
        if not heuristic:
            raise ValueError("Heuristic function is required for Greedy Search.")

        frontier = []  # Fila de prioridade
        heapq.heappush(frontier, (heuristic(start, goal), Node(start)))  # (heurística, nó)
        visited = set()
        nodes_generated, nodes_visited = 0, 0

        while frontier:
            _, current = heapq.heappop(frontier)  # Remove o nó com menor valor heurístico
            nodes_visited += 1

            # Verifica se alcançamos o objetivo
            if current.position == goal:
                return reconstruct_path(current), current.cost, nodes_generated, nodes_visited

            if current.position not in visited:
                visited.add(current.position)

                # Expande os vizinhos
                for action, move in OPERATORS.items():
                    neighbor = move(*current.position)
                    if neighbor and neighbor not in visited:
                        nodes_generated += 1
                        if cost_function in [cost_c3, cost_c4]:
                            new_cost = current.cost + cost_function(action, current.depth + 1)
                        else:
                            new_cost = current.cost + cost_function(action)

                        heapq.heappush(frontier, (heuristic(neighbor, goal), Node(neighbor, current, new_cost, current.depth + 1)))



        return None, float("inf"), nodes_generated, nodes_visited

    elif algorithm == "A*":
      if not heuristic:
          raise ValueError("Heuristic function is required for A* Search.")

      frontier = []  # Fila de prioridade
      heapq.heappush(frontier, (heuristic(start, goal), Node(start)))  # (f(n), nó)
      visited = {}  # Armazena o menor custo para cada nó
      nodes_generated, nodes_visited = 0, 0

      while frontier:
          _, current = heapq.heappop(frontier)  # Remove o nó com menor f(n)
          nodes_visited += 1

          # Verifica se alcançamos o objetivo
          if current.position == goal:
              return reconstruct_path(current), current.cost, nodes_generated, nodes_visited

          # Verifica se devemos explorar o nó (se o custo atual for menor que o registrado)
          if current.position not in visited or current.cost < visited[current.position]:
              visited[current.position] = current.cost

              # Expande os vizinhos
              for action, move in OPERATORS.items():
                  neighbor = move(*current.position)
                  if neighbor:
                      nodes_generated += 1
                      # Verifica se a função de custo requer 'steps'
                      if cost_function in [cost_c3, cost_c4]:
                          g = current.cost + cost_function(action, current.depth + 1)
                      else:
                          g = current.cost + cost_function(action)
                      h = heuristic(neighbor, goal)  # Estimativa da heurística
                      f = g + h  # f(n) = g(n) + h(n)
                      heapq.heappush(frontier, (f, Node(neighbor, current, g, current.depth + 1)))


      return None, float("inf"), nodes_generated, nodes_visited



# Teste os algoritimos

Você pode testar os algorítmos alterando os parâmetros: `start`, `goal` e `cost`.

`start` e `goal` podem receber cordenadas que vão de (0,0) a (30,30)

`cost` pode receber: cost_c1, cost_c2, cost_c3, cost_c4, que representam chamadas para as funções de custo

*Busca em largura*

In [ ]:
# Exemplo de execução
if __name__ == "__main__":
    start = (0, 0)
    goal = (5, 5)
    cost = cost_c3

    path, cost, nodes_gen, nodes_vis = search_algorithm(
        start, goal, "BFS", cost
    )

    print("Caminho:", path)
    print("Custo:", cost)
    print("Nós gerados:", nodes_gen)
    print("Nós visitados:", nodes_vis)


Caminho: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]
Custo: 110
Nós gerados: 71
Nós visitados: 61


*Busca em profundidade*

In [ ]:
# Exemplo de execução
if __name__ == "__main__":
    start = (0, 0)
    goal = (5, 5)
    cost = cost_c3

    path, cost, nodes_gen, nodes_vis = search_algorithm(
        start, goal, "DFS", cost
    )

    print("Caminho:", path)
    print("Custo:", cost)
    print("Nós gerados:", nodes_gen)
    print("Nós visitados:", nodes_vis)


Caminho: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (1, 30), (1, 29), (1, 28), (1, 27), (1, 26), (1, 25), (1, 24), (1, 23), (1, 22), (1, 21), (1, 20), (1, 19), (1, 18), (1, 17), (1, 16), (1, 15), (1, 14), (1, 13), (1, 12), (1, 11), (1, 10), (1, 9), (1, 8), (1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2, 30), (3, 30), (3, 29), (3, 28), (3, 27), (3, 26), (3, 25), (3, 24), (3, 23), (3, 22), (3, 21), (3, 20), (3, 19), (3, 18), (3, 17), (3, 16), (3, 15), (3, 14), (3, 13), (3, 12), (3, 11), (3,

*Busca de custo uniforme*

In [ ]:
# Exemplo de execução
if __name__ == "__main__":
    start = (0, 0)
    goal = (5, 5)
    cost = cost_c1

    path, cost, nodes_gen, nodes_vis = search_algorithm(
        start, goal, "UCS", cost
    )

    print("Caminho:", path)
    print("Custo:", cost)
    print("Nós gerados:", nodes_gen)
    print("Nós visitados:", nodes_vis)


Caminho: [(0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (3, 2), (3, 3), (4, 3), (4, 4), (5, 4), (5, 5)]
Custo: 100
Nós gerados: 227
Nós visitados: 183


*Busca gulosa (heurística Euclidiana)*

In [ ]:
# Exemplo de execução
if __name__ == "__main__":
    start = (0, 0)
    goal = (5, 5)
    cost = cost_c1

    path, cost, nodes_gen, nodes_vis = search_algorithm(
        start, goal, "GREEDY", cost, euclidean_heuristic
    )

    print("Caminho:", path)
    print("Custo:", cost)
    print("Nós gerados:", nodes_gen)
    print("Nós visitados:", nodes_vis)


Caminho: [(0, 0), (1, 0), (1, 1), (2, 1), (2, 2), (3, 2), (3, 3), (4, 3), (4, 4), (5, 4), (5, 5)]
Custo: 100
Nós gerados: 28
Nós visitados: 11


*Busca gulosa (heurística Manhattan)*

In [ ]:
# Exemplo de execução
if __name__ == "__main__":
    start = (0, 0)
    goal = (5, 5)
    cost = cost_c3

    path, cost, nodes_gen, nodes_vis = search_algorithm(
        start, goal, "GREEDY", cost, manhattan_heuristic
    )

    print("Caminho:", path)
    print("Custo:", cost)
    print("Nós gerados:", nodes_gen)
    print("Nós visitados:", nodes_vis)


Caminho: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (1, 5), (2, 5), (3, 5), (4, 5), (5, 5)]
Custo: 114
Nós gerados: 25
Nós visitados: 11


*A * (heurística Euclidiana)*

In [ ]:
# Exemplo de execução
if __name__ == "__main__":
    start = (0, 0)
    goal = (5, 5)
    cost = cost_c1

    path, cost, nodes_gen, nodes_vis = search_algorithm(
        start, goal, "A*", cost, euclidean_heuristic
    )

    print("Caminho:", path)
    print("Custo:", cost)
    print("Nós gerados:", nodes_gen)
    print("Nós visitados:", nodes_vis)


Caminho: [(0, 0), (1, 0), (1, 1), (2, 1), (3, 1), (3, 2), (3, 3), (4, 3), (4, 4), (5, 4), (5, 5)]
Custo: 100
Nós gerados: 128
Nós visitados: 76


*A * (Heurística Manhattan)*

In [ ]:
# Exemplo de execução
if __name__ == "__main__":
    start = (0, 0)
    goal = (5, 5)
    cost = cost_c1

    path, cost, nodes_gen, nodes_vis = search_algorithm(
        start, goal, "A*", cost, manhattan_heuristic
    )

    print("Caminho:", path)
    print("Custo:", cost)
    print("Nós gerados:", nodes_gen)
    print("Nós visitados:", nodes_vis)


Caminho: [(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (4, 4), (5, 4), (5, 5)]
Custo: 100
Nós gerados: 128
Nós visitados: 60


A* (Heurística Manhattan) randomizado

In [ ]:
# Exemplo de execução
if __name__ == "__main__":
    start = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))
    goal = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))

    path, cost, nodes_gen, nodes_vis = search_algorithm(
        start, goal, "A*", cost_c1, manhattan_heuristic
    )

    print("Caminho:", path)
    print("Custo:", cost)
    print("Nós gerados:", nodes_gen)
    print("Nós visitados:", nodes_vis)


Caminho: [(3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (8, 3), (9, 3), (9, 4), (9, 5), (10, 5), (11, 5), (12, 5), (12, 6), (13, 6), (14, 6), (14, 7), (14, 8), (14, 9), (14, 10), (14, 11), (15, 11), (15, 12), (15, 13)]
Custo: 230
Nós gerados: 620
Nós visitados: 287


# Parte 1: Largura vs. Profundidade vs. Custo Uniforme

In [ ]:
def run_experiment_part1():
    contador = 0
    results = []
    for _ in range(50):
        start = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))
        goal = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))

        for algorithm in ["BFS", "DFS", "UCS"]:
            for cost_function in [cost_c1, cost_c2, cost_c3, cost_c4]:
                contador+=1
                path, cost, nodes_gen, nodes_vis = search_algorithm(start, goal, algorithm, cost_function)
                results.append({
                    "n° registro": contador,
                    "start": start,
                    "goal": goal,
                    "algorithm": algorithm,
                    "cost_function": cost_function.__name__,
                    "path": path,
                    "cost": cost,
                    "nodes_generated": nodes_gen,
                    "nodes_visited": nodes_vis
                })

    # Grava os resultados em um arquivo CSV
    with open("experiment_csv/experiment_part1_results.csv", "w", newline="") as csvfile:
        fieldnames = [
            "n° registro", "start", "goal", "algorithm",
            "cost_function", "path", "cost",
            "nodes_generated", "nodes_visited"
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for result in results:
            writer.writerow({
                "n° registro": result["n° registro"],
                "start": result["start"],
                "goal": result["goal"],
                "algorithm": result["algorithm"],
                "cost_function": result["cost_function"],
                "path": result["path"],
                "cost": result["cost"],
                "nodes_generated": result["nodes_generated"],
                "nodes_visited": result["nodes_visited"]
            })


    with open("experiment_txt/experiment_part1_results.txt", "w") as f:
        for result in results:
            f.write(f"n°: {result['n° registro']}\n")
            f.write(f"Start: {result['start']}\n")
            f.write(f"Goal: {result['goal']}\n")
            f.write(f"Algorithm: {result['algorithm']}\n")
            f.write(f"Cost Function: {result['cost_function']}\n")
            f.write(f"Path: {result['path']}\n")
            f.write(f"Cost: {result['cost']}\n")
            f.write(f"Nodes Generated: {result['nodes_generated']}\n")
            f.write(f"Nodes Visited: {result['nodes_visited']}\n")
            f.write("-" * 40 + "\n")

In [ ]:
run_experiment_part1()

# Parte 2: Custo Uniforme vs A*

In [ ]:
def run_experiment_part2():
    contador = 0
    results = []

    for _ in range(50):  # 50 pares de start e goal
        start = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))
        goal = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))

        for algorithm in ["UCS", "A*"]:
            for cost_function in [cost_c1, cost_c2, cost_c3, cost_c4]:
                if algorithm == "A*":
                    for heuristic in [euclidean_heuristic, manhattan_heuristic]:
                        contador += 1
                        path, cost, nodes_gen, nodes_vis = search_algorithm(start, goal, algorithm, cost_function, heuristic)
                        results.append({
                            "n° registro": contador,
                            "start": start,
                            "goal": goal,
                            "algorithm": algorithm,
                            "cost_function": cost_function.__name__,
                            "heuristic": heuristic.__name__,
                            "path": path,
                            "cost": cost,
                            "nodes_generated": nodes_gen,
                            "nodes_visited": nodes_vis
                        })
                else:  # UCS não usa heurística
                    contador += 1
                    path, cost, nodes_gen, nodes_vis = search_algorithm(start, goal, algorithm, cost_function)
                    results.append({
                        "n° registro": contador,
                        "start": start,
                        "goal": goal,
                        "algorithm": algorithm,
                        "cost_function": cost_function.__name__,
                        "heuristic": None,
                        "path": path,
                        "cost": cost,
                        "nodes_generated": nodes_gen,
                        "nodes_visited": nodes_vis
                    })

    with open("experiment_csv/experiment_part2_results.csv", "w", newline="") as csvfile:
        fieldnames = [
            "n° registro", "start", "goal", "algorithm",
            "cost_function", "heuristic", "path",
            "cost", "nodes_generated", "nodes_visited"
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for result in results:
            writer.writerow({
                "n° registro": result["n° registro"],
                "start": result["start"],
                "goal": result["goal"],
                "algorithm": result["algorithm"],
                "cost_function": result["cost_function"],
                "heuristic": result["heuristic"],
                "path": result["path"],
                "cost": result["cost"],
                "nodes_generated": result["nodes_generated"],
                "nodes_visited": result["nodes_visited"]
            })

    with open("experiment_txt/experiment_part2_results.txt", "w") as f:
        for result in results:
            f.write(f"n°: {result['n° registro']}\n")
            f.write(f"Start: {result['start']}\n")
            f.write(f"Goal: {result['goal']}\n")
            f.write(f"Algorithm: {result['algorithm']}\n")
            f.write(f"Cost Function: {result['cost_function']}\n")
            if result['heuristic']:
                f.write(f"Heuristic: {result['heuristic']}\n")
            f.write(f"Path: {result['path']}\n")
            f.write(f"Cost: {result['cost']}\n")
            f.write(f"Nodes Generated: {result['nodes_generated']}\n")
            f.write(f"Nodes Visited: {result['nodes_visited']}\n")
            f.write("-" * 40 + "\n")


In [ ]:
run_experiment_part2()

# Parte 3: Gulosa vs A*

In [ ]:
def run_experiment_part3():
    contador = 0
    results = []

    for _ in range(50):  # 50 pares de start e goal
        start = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))
        goal = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))

        for algorithm in ["GREEDY", "A*"]:
            for cost_function in [cost_c1, cost_c2, cost_c3, cost_c4]:
                for heuristic in [euclidean_heuristic, manhattan_heuristic]:
                    contador += 1
                    path, cost, nodes_gen, nodes_vis = search_algorithm(start, goal, algorithm, cost_function, heuristic)
                    results.append({
                        "n° registro": contador,
                        "start": start,
                        "goal": goal,
                        "algorithm": algorithm,
                        "cost_function": cost_function.__name__,
                        "heuristic": heuristic.__name__,
                        "path": path,
                        "cost": cost,
                        "nodes_generated": nodes_gen,
                        "nodes_visited": nodes_vis
                    })


    # Grava os resultados em um arquivo CSV
    with open("experiment_csv/experiment_part3_results.csv", "w", newline="") as csvfile:
        fieldnames = [
            "n° registro", "start", "goal", "algorithm",
            "cost_function", "heuristic", "path",
            "cost", "nodes_generated", "nodes_visited"
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for result in results:
            writer.writerow({
                "n° registro": result["n° registro"],
                "start": result["start"],
                "goal": result["goal"],
                "algorithm": result["algorithm"],
                "cost_function": result["cost_function"],
                "heuristic": result["heuristic"],
                "path": result["path"],
                "cost": result["cost"],
                "nodes_generated": result["nodes_generated"],
                "nodes_visited": result["nodes_visited"]
            })

    with open("experiment_txt/experiment_part3_results.txt", "w") as f:
        for result in results:
            f.write(f"n°: {result['n° registro']}\n")
            f.write(f"Start: {result['start']}\n")
            f.write(f"Goal: {result['goal']}\n")
            f.write(f"Algorithm: {result['algorithm']}\n")
            f.write(f"Cost Function: {result['cost_function']}\n")
            f.write(f"Heuristic: {result['heuristic']}\n")
            f.write(f"Path: {result['path']}\n")
            f.write(f"Cost: {result['cost']}\n")
            f.write(f"Nodes Generated: {result['nodes_generated']}\n")
            f.write(f"Nodes Visited: {result['nodes_visited']}\n")
            f.write("-" * 40 + "\n")




In [ ]:
run_experiment_part3()

# Parte 4: Largura vs. Profundidade com Randomização da Vizinhança

In [ ]:
def run_experiment_part4():
    global OPERATORS
    contador = 0
    results = []

    for _ in range(20):  # 20 pares de start e goal
        start = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))
        goal = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))

        for algorithm in ["BFS", "DFS"]:
            for cost_function in [cost_c1, cost_c2, cost_c3, cost_c4]:

                # Embaralha a ordem dos operadores para cada execução
                shuffled_operators = list(OPERATORS.items())
                random.shuffle(shuffled_operators)
                OPERATORS = dict(shuffled_operators)

                # Executa 10 vezes o algoritmo com a randomização dos vizinhos
                for _ in range(10):
                    contador += 1
                    path, cost, nodes_gen, nodes_vis = search_algorithm(start, goal, algorithm, cost_function)

                    # Armazena o resultado
                    results.append({
                        "n° registro": contador,
                        "start": start,
                        "goal": goal,
                        "algorithm": algorithm,
                        "cost_function": cost_function.__name__,
                        "path": path,
                        "cost": cost,
                        "nodes_generated": nodes_gen,
                        "nodes_visited": nodes_vis
                    })

    # Salvar os resultados em um arquivo CSV
    with open("experiment_csv/experiment_part4_results.csv", "w", newline="") as csvfile:
        fieldnames = [
            "n° registro", "start", "goal", "algorithm",
            "cost_function", "path", "cost",
            "nodes_generated", "nodes_visited"
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for result in results:
            writer.writerow({
                "n° registro": result["n° registro"],
                "start": result["start"],
                "goal": result["goal"],
                "algorithm": result["algorithm"],
                "cost_function": result["cost_function"],
                "path": result["path"],
                "cost": result["cost"],
                "nodes_generated": result["nodes_generated"],
                "nodes_visited": result["nodes_visited"]
            })

    with open("experiment_txt/experiment_part4_results.txt", "w") as file:
        for result in results:
            file.write(f"n°: {result['n° registro']}\n")
            file.write(f"Start: {result['start']}\n")
            file.write(f"Goal: {result['goal']}\n")
            file.write(f"Algorithm: {result['algorithm']}\n")
            file.write(f"Cost Function: {result['cost_function']}\n")
            file.write(f"Path: {result['path']}\n")
            file.write(f"Cost: {result['cost']}\n")
            file.write(f"Nodes Generated: {result['nodes_generated']}\n")
            file.write(f"Nodes Visited: {result['nodes_visited']}\n")
            file.write("-" * 40 + "\n")

In [ ]:
run_experiment_part4()

# Parte 5: Caminho Mínimo Com Uma Parada A Mais

In [ ]:
def run_experiment_part5():
    contador = 0
    results = []

    for _ in range(25):  # 25 experimentos
        # Gerar pontos aleatórios: trabalho (start), casa (goal) e 4 farmácias
        start = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))  # trabalho
        goal = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))  # casa
        pharmacies = set()

        while len(pharmacies) < 4:  # Garantir que não haja farmácias repetidas
            pharmacy = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))
            pharmacies.add(pharmacy)

        pharmacies = list(pharmacies)

        # Execute a Busca A* com as combinações de funções de custo e heurísticas
        for cost_function in [cost_c1, cost_c2, cost_c3, cost_c4]:
            for heuristic in [euclidean_heuristic, manhattan_heuristic]:
                best_path = None
                best_cost = float('inf')
                best_nodes_gen = 0
                best_nodes_vis = 0
                best_pharmacy = None

                # Avaliar o caminho passando por cada farmácia
                for pharmacy in pharmacies:
                    # Caminho de start → farmácia
                    path1, cost1, nodes_gen1, nodes_vis1 = search_algorithm(start, pharmacy, "A*", cost_function, heuristic)
                    # Caminho de farmácia → goal
                    path2, cost2, nodes_gen2, nodes_vis2 = search_algorithm(pharmacy, goal, "A*", cost_function, heuristic)

                    # Calcular o custo total
                    total_cost = cost1 + cost2
                    total_nodes_gen = nodes_gen1 + nodes_gen2
                    total_nodes_vis = nodes_vis1 + nodes_vis2

                    # Verificar se esse caminho é melhor
                    if total_cost < best_cost:
                        best_cost = total_cost
                        best_path = path1 + path2[1:]  # Concatenar os caminhos sem repetir a farmácia
                        best_nodes_gen = total_nodes_gen
                        best_nodes_vis = total_nodes_vis
                        best_pharmacy = pharmacy

                contador += 1

                # Armazenar o resultado do melhor caminho
                results.append({
                    "n° registro": contador,
                    "start": start,
                    "goal": goal,
                    "chosen_pharmacy": best_pharmacy,
                    "pharmacies": pharmacies,
                    "cost_function": cost_function.__name__,
                    "heuristic": heuristic.__name__,
                    "path": best_path,
                    "cost": best_cost,
                    "nodes_generated": best_nodes_gen,
                    "nodes_visited": best_nodes_vis
                })

    # Salvar os resultados em um arquivo CSV
    with open("experiment_csv/experiment_part5_results.csv", "w", newline="") as csvfile:
        fieldnames = [
            "n° registro", "start", "goal", "chosen_pharmacy", "pharmacies",
            "cost_function", "heuristic", "path", "cost",
            "nodes_generated", "nodes_visited"
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for result in results:
            writer.writerow({
                "n° registro": result["n° registro"],
                "start": result["start"],
                "goal": result["goal"],
                "chosen_pharmacy": result["chosen_pharmacy"],
                "pharmacies": result["pharmacies"],
                "cost_function": result["cost_function"],
                "heuristic": result["heuristic"],
                "path": result["path"],
                "cost": result["cost"],
                "nodes_generated": result["nodes_generated"],
                "nodes_visited": result["nodes_visited"]
            })

    # Salvar os resultados em um arquivo TXT
    with open("experiment_txt/experiment_part5_results.txt", "w") as file:
        for result in results:
            file.write(f"n°: {result['n° registro']}\n")
            file.write(f"Start: {result['start']}\n")
            file.write(f"Goal: {result['goal']}\n")
            file.write(f"Chosen Pharmacy: {result['chosen_pharmacy']}\n")
            file.write(f"Pharmacies: {result['pharmacies']}\n")
            file.write(f"Cost Function: {result['cost_function']}\n")
            file.write(f"Heuristic: {result['heuristic']}\n")
            file.write(f"Path: {result['path']}\n")
            file.write(f"Cost: {result['cost']}\n")
            file.write(f"Nodes Generated: {result['nodes_generated']}\n")
            file.write(f"Nodes Visited: {result['nodes_visited']}\n")
            file.write("-" * 40 + "\n")


In [ ]:
run_experiment_part5()

# Parte 0

In [ ]:
def run_experiment_part0():
    contador = 0
    results = []

    for _ in range(20):  # 20 pares de start e goal
        start = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))
        goal = (random.randint(0, MAP_LIMIT), random.randint(0, MAP_LIMIT))

        for algorithm in ["BFS", "DFS", "UCS", "GREEDY", "A*"]:
            for cost_function in [cost_c1, cost_c2, cost_c3, cost_c4]:
                # Define heurísticas disponíveis (para GREEDY e A*)
                heuristics = (
                    [None] if algorithm in ["BFS", "DFS", "UCS"]
                    else [euclidean_heuristic, manhattan_heuristic]
                )
                for heuristic in heuristics:
                    contador += 1

                    # Executa o algoritmo de busca
                    path, cost, nodes_gen, nodes_vis = search_algorithm(
                        start, goal, algorithm, cost_function, heuristic
                    )

                    # Armazena os resultados
                    results.append({
                        "n° registro": contador,
                        "start": start,
                        "goal": goal,
                        "algorithm": algorithm,
                        "cost_function": cost_function.__name__,
                        "heuristic": heuristic.__name__ if heuristic else "None",
                        "path": path,
                        "cost": cost,
                        "nodes_generated": nodes_gen,
                        "nodes_visited": nodes_vis,
                    })

    # Grava os resultados em um arquivo CSV
    with open("experiment_csv/experiment_part0_results.csv", "w", newline="") as csvfile:
        fieldnames = [
            "n° registro", "start", "goal", "algorithm",
            "cost_function", "heuristic", "path", "cost",
            "nodes_generated", "nodes_visited"
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for result in results:
            writer.writerow(result)

    # Salvar os resultados em um arquivo TXT
    with open("experiment_txt/experiment_part0_results.txt", "w") as file:
        for result in results:
            file.write(f"n°: {result['n° registro']}\n")
            file.write(f"Start: {result['start']}\n")
            file.write(f"Goal: {result['goal']}\n")
            file.write(f"Algorithm: {result['algorithm']}\n")
            file.write(f"Cost Function: {result['cost_function']}\n")
            file.write(f"Heuristic: {result['heuristic']}\n")
            file.write(f"Path: {result['path']}\n")
            file.write(f"Cost: {result['cost']}\n")
            file.write(f"Nodes Generated: {result['nodes_generated']}\n")
            file.write(f"Nodes Visited: {result['nodes_visited']}\n")
            file.write("-" * 40 + "\n")


In [ ]:
run_experiment_part0()

# Gráficos

In [ ]:
def plot_map(path, title, ax):
    ax.set_title(title)
    ax.set_xlabel('Eixo X')
    ax.set_ylabel('Eixo Y')
    ax.grid(True)
    ax.set_xlim(0, 30)
    ax.set_ylim(0, 30)

    # Eixos x e y de 1 em 1
    ax.set_xticks(range(0, 31, 1))
    ax.set_yticks(range(0, 31, 1))

    ax.tick_params(axis='x', labelsize=6)  # Ajuste o tamanho da fonte para os marcadores do eixo X
    ax.tick_params(axis='y', labelsize=6)

    start = path[0]  # Ponto inicial
    goal = path[-1]  # Ponto final

    # Trajetória
    ax.plot([p[0] for p in path], [p[1] for p in path], 'ro-')

    # Marcador do início (triângulo azul)
    ax.scatter(start[0], start[1], color='blue', marker='^', s=150, label='Início', zorder=2)

    # Marcador do fim (estrela verde)
    ax.scatter(goal[0], goal[1], color='green', marker='*', s=150, label='Fim', zorder=2)

    ax.legend()  # Adiciona legenda


In [ ]:
def plot_bar_chart(costList, labels, title, ax):
    bars = ax.bar(range(len(costList)), costList, color='skyblue')  # Plota as barras

    # Adicionar título e rótulos
    ax.set_title(title, fontsize=14)
    ax.set_xlabel('Algoritmo/Função de Custo', fontsize=10)
    ax.set_ylabel('Custo', fontsize=10)

    # Definir os rótulos no eixo X
    ax.set_xticks(range(len(costList)))
    ax.set_xticklabels(labels, rotation=45)  # Ajusta os rótulos para cada barra

    # Adicionar valores acima das barras
    for bar in bars:
        yval = bar.get_height()  # Altura de cada barra (o valor do custo)
        ax.text(bar.get_x() + bar.get_width() / 2, yval, str(yval), ha='center', va='bottom', fontsize=10)

    ax.grid(axis='y', linestyle='--', alpha=0.7)

In [ ]:
# Solicitar ao usuário o número do intervalo (1 a 50)
def solicitar_intervalo(rng):
    while True:
        try:
            intervalo_numero = int(input(f"Digite o número do intervalo (de 1 a {rng}): "))  # Solicita o número
            if 1 <= intervalo_numero <= rng:  # Verifica se o número está dentro do intervalo
                return intervalo_numero  # Retorna o número se estiver dentro do intervalo
            else:
                print(f"Número fora do intervalo permitido! Tente novamente entre 1 e {rng}.")
        except ValueError:  # Caso o valor não seja um número válido
            print(f"Entrada inválida! Digite um número inteiro entre 1 e {rng}.")

# Função para obter o intervalo e acessar a coluna 'path'
def obter_intervalo_e_path(dataFramePai, intervalo_numero, numero_de_intervalos, tamanho_intervalos):
    if 1 <= intervalo_numero <= numero_de_intervalos:
        inicio = (intervalo_numero - 1) * tamanho_intervalos
        fim = inicio + tamanho_intervalos
        intervalo = dataFramePai.iloc[inicio:fim]
        return intervalo
    else:
        print("Número de intervalo inválido. Digite um número entre 1 e 50.")
        return None

# Análise dos resultados

## Parte 1

In [ ]:
df_1 = pd.read_csv("experiment_csv/experiment_part1_results.csv")
df_1.index = df_1.index + 1

df_1_experimento = obter_intervalo_e_path(df_1, solicitar_intervalo(50),50,12)
df_1_experimento

Digite o número do intervalo (de 1 a 50): 1


,n° registro,start,goal,algorithm,cost_function,path,cost,nodes_generated,nodes_visited
1,1,"(1, 7)","(11, 22)",BFS,cost_c1,"[(1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7...",250,555,525
2,2,"(1, 7)","(11, 22)",BFS,cost_c2,"[(1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7...",300,555,525
3,3,"(1, 7)","(11, 22)",BFS,cost_c3,"[(1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7...",275,555,525
4,4,"(1, 7)","(11, 22)",BFS,cost_c4,"[(1, 7), (2, 7), (3, 7), (4, 7), (5, 7), (6, 7...",245,555,525
5,5,"(1, 7)","(11, 22)",DFS,cost_c1,"[(1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2...",2950,612,296
6,6,"(1, 7)","(11, 22)",DFS,cost_c2,"[(1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2...",3000,612,296
7,7,"(1, 7)","(11, 22)",DFS,cost_c3,"[(1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2...",2977,612,296
8,8,"(1, 7)","(11, 22)",DFS,cost_c4,"[(1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 2...",2948,612,296
9,9,"(1, 7)","(11, 22)",UCS,cost_c1,"[(1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1,...",250,2057,1929
10,10,"(1, 7)","(11, 22)",UCS,cost_c2,"[(1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1,...",300,1802,1696


## Parte 2

In [ ]:
df_2 = pd.read_csv("experiment_csv/experiment_part2_results.csv")
df_2.index = df_2.index + 1

df_2_experimento = obter_intervalo_e_path(df_2, solicitar_intervalo(50),50,12)
df_2_experimento

Digite o número do intervalo (de 1 a 50): 1


,n° registro,start,goal,algorithm,cost_function,heuristic,path,cost,nodes_generated,nodes_visited
1,1,"(0, 20)","(27, 9)",UCS,cost_c1,NaN,"[(0, 20), (1, 20), (2, 20), (3, 20), (4, 20), ...",380,3412,3323
2,2,"(0, 20)","(27, 9)",UCS,cost_c2,NaN,"[(0, 20), (1, 20), (2, 20), (3, 20), (4, 20), ...",515,3419,3354
3,3,"(0, 20)","(27, 9)",UCS,cost_c3,NaN,"[(0, 20), (0, 19), (1, 19), (2, 19), (3, 19), ...",422,3415,3346
4,4,"(0, 20)","(27, 9)",UCS,cost_c4,NaN,"[(0, 20), (0, 19), (0, 18), (0, 17), (1, 17), ...",329,3399,3330
5,5,"(0, 20)","(27, 9)",A*,cost_c1,euclidean_heuristic,"[(0, 20), (1, 20), (2, 20), (3, 20), (4, 20), ...",380,1690,1411
6,6,"(0, 20)","(27, 9)",A*,cost_c1,manhattan_heuristic,"[(0, 20), (1, 20), (2, 20), (3, 20), (4, 20), ...",380,1328,632
7,7,"(0, 20)","(27, 9)",A*,cost_c2,euclidean_heuristic,"[(0, 20), (1, 20), (2, 20), (3, 20), (4, 20), ...",515,2593,2401
8,8,"(0, 20)","(27, 9)",A*,cost_c2,manhattan_heuristic,"[(0, 20), (1, 20), (2, 20), (3, 20), (4, 20), ...",515,1988,1752
9,9,"(0, 20)","(27, 9)",A*,cost_c3,euclidean_heuristic,"[(0, 20), (0, 19), (1, 19), (2, 19), (3, 19), ...",422,1961,1730
10,10,"(0, 20)","(27, 9)",A*,cost_c3,manhattan_heuristic,"[(0, 20), (0, 19), (1, 19), (2, 19), (3, 19), ...",422,1260,941


In [ ]:
# 8exec A Star

## Parte 3

In [ ]:
df_3 = pd.read_csv("experiment_csv/experiment_part3_results.csv")
df_3.index = df_3.index + 1

df_3_experimento = obter_intervalo_e_path(df_3,solicitar_intervalo(50),50,16)
df_3_experimento

Digite o número do intervalo (de 1 a 50): 1


,n° registro,start,goal,algorithm,cost_function,heuristic,path,cost,nodes_generated,nodes_visited
1,1,"(26, 18)","(22, 22)",GREEDY,cost_c1,euclidean_heuristic,"[(26, 18), (25, 18), (25, 19), (24, 19), (24, ...",80,25,9
2,2,"(26, 18)","(22, 22)",GREEDY,cost_c1,manhattan_heuristic,"[(26, 18), (25, 18), (24, 18), (23, 18), (22, ...",80,25,9
3,3,"(26, 18)","(22, 22)",GREEDY,cost_c2,euclidean_heuristic,"[(26, 18), (26, 19), (25, 19), (25, 20), (24, ...",100,25,9
4,4,"(26, 18)","(22, 22)",GREEDY,cost_c2,manhattan_heuristic,"[(26, 18), (26, 19), (26, 20), (26, 21), (26, ...",100,25,9
5,5,"(26, 18)","(22, 22)",GREEDY,cost_c3,euclidean_heuristic,"[(26, 18), (26, 19), (25, 19), (25, 20), (24, ...",87,25,9
6,6,"(26, 18)","(22, 22)",GREEDY,cost_c3,manhattan_heuristic,"[(26, 18), (26, 19), (26, 20), (26, 21), (26, ...",86,25,9
7,7,"(26, 18)","(22, 22)",GREEDY,cost_c4,euclidean_heuristic,"[(26, 18), (26, 19), (25, 19), (25, 20), (24, ...",82,25,9
8,8,"(26, 18)","(22, 22)",GREEDY,cost_c4,manhattan_heuristic,"[(26, 18), (26, 19), (26, 20), (26, 21), (26, ...",74,25,9
9,9,"(26, 18)","(22, 22)",A*,cost_c1,euclidean_heuristic,"[(26, 18), (25, 18), (24, 18), (23, 18), (23, ...",80,112,56
10,10,"(26, 18)","(22, 22)",A*,cost_c1,manhattan_heuristic,"[(26, 18), (26, 19), (26, 20), (26, 21), (25, ...",80,96,40


## Parte 4

In [ ]:
df_4 = pd.read_csv("experiment_csv/experiment_part4_results.csv")
df_4.index = df_4.index + 1

df_4_experimento = obter_intervalo_e_path(df_4, solicitar_intervalo(50),50,80)
df_4_experimento

Digite o número do intervalo (de 1 a 50): 1


,n° registro,start,goal,algorithm,cost_function,path,cost,nodes_generated,nodes_visited
1,1,"(4, 18)","(29, 29)",BFS,cost_c1,"[(4, 18), (4, 19), (4, 20), (4, 21), (4, 22), ...",360,930,921
2,2,"(4, 18)","(29, 29)",BFS,cost_c1,"[(4, 18), (4, 19), (4, 20), (4, 21), (4, 22), ...",360,930,921
3,3,"(4, 18)","(29, 29)",BFS,cost_c1,"[(4, 18), (4, 19), (4, 20), (4, 21), (4, 22), ...",360,930,921
4,4,"(4, 18)","(29, 29)",BFS,cost_c1,"[(4, 18), (4, 19), (4, 20), (4, 21), (4, 22), ...",360,930,921
5,5,"(4, 18)","(29, 29)",BFS,cost_c1,"[(4, 18), (4, 19), (4, 20), (4, 21), (4, 22), ...",360,930,921
...,...,...,...,...,...,...,...,...,...
76,76,"(4, 18)","(29, 29)",DFS,cost_c4,"[(4, 18), (4, 19), (4, 20), (4, 21), (4, 22), ...",1108,219,111
77,77,"(4, 18)","(29, 29)",DFS,cost_c4,"[(4, 18), (4, 19), (4, 20), (4, 21), (4, 22), ...",1108,219,111
78,78,"(4, 18)","(29, 29)",DFS,cost_c4,"[(4, 18), (4, 19), (4, 20), (4, 21), (4, 22), ...",1108,219,111
79,79,"(4, 18)","(29, 29)",DFS,cost_c4,"[(4, 18), (4, 19), (4, 20), (4, 21), (4, 22), ...",1108,219,111


## Parte 5

In [ ]:
df_5 = pd.read_csv("experiment_csv/experiment_part5_results.csv")
df_5.index = df_5.index + 1

df_5_experimento = obter_intervalo_e_path(df_5,solicitar_intervalo(50),50,8)
df_5_experimento

Digite o número do intervalo (de 1 a 50): 1


,n° registro,start,goal,chosen_pharmacy,pharmacies,cost_function,heuristic,path,cost,nodes_generated,nodes_visited
1,1,"(9, 14)","(30, 10)","(5, 12)","[(4, 4), (5, 5), (5, 12), (3, 10)]",cost_c1,euclidean_heuristic,"[(9, 14), (8, 14), (7, 14), (6, 14), (6, 13), ...",330,454,193
2,2,"(9, 14)","(30, 10)","(5, 12)","[(4, 4), (5, 5), (5, 12), (3, 10)]",cost_c1,manhattan_heuristic,"[(9, 14), (8, 14), (7, 14), (6, 14), (6, 13), ...",330,362,149
3,3,"(9, 14)","(30, 10)","(5, 12)","[(4, 4), (5, 5), (5, 12), (3, 10)]",cost_c2,euclidean_heuristic,"[(9, 14), (8, 14), (7, 14), (6, 14), (6, 13), ...",475,1688,1426
4,4,"(9, 14)","(30, 10)","(5, 12)","[(4, 4), (5, 5), (5, 12), (3, 10)]",cost_c2,manhattan_heuristic,"[(9, 14), (9, 13), (8, 13), (7, 13), (6, 13), ...",475,1091,890
5,5,"(9, 14)","(30, 10)","(5, 12)","[(4, 4), (5, 5), (5, 12), (3, 10)]",cost_c3,euclidean_heuristic,"[(9, 14), (9, 13), (9, 12), (8, 12), (7, 12), ...",389,1016,760
6,6,"(9, 14)","(30, 10)","(5, 12)","[(4, 4), (5, 5), (5, 12), (3, 10)]",cost_c3,manhattan_heuristic,"[(9, 14), (9, 13), (9, 12), (8, 12), (7, 12), ...",389,576,411
7,7,"(9, 14)","(30, 10)","(5, 12)","[(4, 4), (5, 5), (5, 12), (3, 10)]",cost_c4,euclidean_heuristic,"[(9, 14), (9, 13), (9, 12), (8, 12), (7, 12), ...",317,242,81
8,8,"(9, 14)","(30, 10)","(5, 12)","[(4, 4), (5, 5), (5, 12), (3, 10)]",cost_c4,manhattan_heuristic,"[(9, 14), (9, 13), (9, 12), (8, 12), (7, 12), ...",311,140,39


## Parte 0

In [ ]:
df_0 = pd.read_csv("experiment_csv/experiment_part0_results.csv")
df_0.index = df_0.index + 1

df_0_experimento = obter_intervalo_e_path(df_0, solicitar_intervalo(20),20,28)
df_0_experimento

Digite o número do intervalo (de 1 a 20): 1


,n° registro,start,goal,algorithm,cost_function,heuristic,path,cost,nodes_generated,nodes_visited
1,1,"(21, 19)","(14, 19)",BFS,cost_c1,NaN,"[(21, 19), (20, 19), (19, 19), (18, 19), (17, ...",70,112,86
2,2,"(21, 19)","(14, 19)",BFS,cost_c2,NaN,"[(21, 19), (20, 19), (19, 19), (18, 19), (17, ...",105,112,86
3,3,"(21, 19)","(14, 19)",BFS,cost_c3,NaN,"[(21, 19), (20, 19), (19, 19), (18, 19), (17, ...",83,112,86
4,4,"(21, 19)","(14, 19)",BFS,cost_c4,NaN,"[(21, 19), (20, 19), (19, 19), (18, 19), (17, ...",77,112,86
5,5,"(21, 19)","(14, 19)",DFS,cost_c1,NaN,"[(21, 19), (21, 20), (21, 21), (21, 22), (21, ...",2490,1014,726
6,6,"(21, 19)","(14, 19)",DFS,cost_c2,NaN,"[(21, 19), (21, 20), (21, 21), (21, 22), (21, ...",2535,1014,726
7,7,"(21, 19)","(14, 19)",DFS,cost_c3,NaN,"[(21, 19), (21, 20), (21, 21), (21, 22), (21, ...",2511,1014,726
8,8,"(21, 19)","(14, 19)",DFS,cost_c4,NaN,"[(21, 19), (21, 20), (21, 21), (21, 22), (21, ...",2485,1014,726
9,9,"(21, 19)","(14, 19)",UCS,cost_c1,NaN,"[(21, 19), (20, 19), (19, 19), (18, 19), (17, ...",70,368,260
10,10,"(21, 19)","(14, 19)",UCS,cost_c2,NaN,"[(21, 19), (20, 19), (19, 19), (18, 19), (17, ...",105,616,478
